In [ ]:
#in this file, the original data which was extracted from SEC is used, contained in a file risks.csv
#the data in the 1A risk factor column is preprocessed and cleaned
#the 1A risk factor is then split into paragraphs and the rows are exploded
#the one's having a word count of less than 20 in the split risk factor column are removed
#the resulting dataframe is then saved in a csv file
#the data in the column '1a_split_finalProcessed' can be used for topic modelling
#'1a_split_finalProcessed' is a tokenized list and can be passed directly into an LDA model


In [1]:
import pandas as pd
df=pd.read_csv(r'risks.csv',encoding="utf-8")

In [2]:
df.head(1)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,...,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate,10k_risks1A,10k_risks7,10k_risks7A
0,9b58a0b3d70dc9de06bdf3d560b3317a,0000320193-22-000108,320193,AAPL,Apple Inc.,Apple Inc. (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2022-10-27T18:01:14-04:00,https://www.sec.gov/Archives/edgar/data/320193...,...,https://www.sec.gov/Archives/edgar/data/320193...,"[{'companyName': 'Apple Inc. (Filer)', 'cik': ...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '8', 'description': 'XBRL TAXONO...",[],2022-09-24,NaN,Item 1A. Risk Factors \n\nThe Company&#8217;s...,Item 7. Management&#8217;s Discussion and Ana...,Item 7A. Quantitative and Qualitative Disclos...


In [3]:
df.shape

(77346, 22)

In [5]:
df.nunique()

id                                      70029
accessionNo                             69800
cik                                      6704
ticker                                   6173
companyName                              9733
companyNameLong                          9733
formType                                    1
description                                 5
filedAt                                 69543
linkToTxt                               70029
linkToHtml                              70008
linkToXbrl                                  0
linkToFilingDetails                     69800
entities                                69852
documentFormatFiles                     69850
dataFiles                               48569
seriesAndClassesContractsInformation        1
periodOfReport                            799
effectivenessDate                         173
10k_risks1A                             60775
10k_risks7                              67785
10k_risks7A                       

In [6]:
#remove the rows with repetation wrt linkToFilingDetails
df.drop_duplicates(subset='linkToFilingDetails', inplace=True)

In [7]:
df.shape

(69800, 22)

In [15]:
df.isna().sum()

id                                          0
accessionNo                                 0
cik                                         0
ticker                                     41
companyName                                 0
companyNameLong                             0
formType                                    0
description                                 0
filedAt                                     0
linkToTxt                                   0
linkToHtml                                  0
linkToXbrl                              69800
linkToFilingDetails                         0
entities                                    0
documentFormatFiles                         0
dataFiles                                   0
seriesAndClassesContractsInformation    14581
periodOfReport                          20904
effectivenessDate                       64203
10k_risks1A                              6460
10k_risks7                               1181
10k_risks7A                       

In [22]:
risk_1a=list(df['10k_risks1A'])

In [23]:
#list of 1a risks
print(len(risk_1a))
print(type(risk_1a))

63340
<class 'list'>


In [16]:
#remove all the rows which have null values in the column "10k_risks1A"

df = df.dropna(axis=0, subset=['10k_risks1A'])

In [17]:
df.shape

(63340, 22)

In [19]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

nltk.download("stopwords")
nltk.download("punkt")
#stop=list('liabilities transactions acquisitions accounts risk risks companys company future factors required tablestart tableend able obtain additional us unable would newcompany will value information years upon company’s fiscal rate based report sales management services form costs related tax ended certain market credit products amount period net including operations securities cash time statements income section common assets shares business plan year date interest december agreement stock may financial million shall also result could results significant including tableendtablestart'.split(" "))
#stop_words = set(stopwords.words("english")+stop)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\partha.mohapatra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\partha.mohapatra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
stopdate=""

In [24]:
#removing particular phrases from a list of string
import re

def remove_phrases_from_list(list_of_strings, phrases_to_remove):
    cleaned_list = []
    for text in list_of_strings:
        cleaned_text = text
        for phrase in phrases_to_remove:
            cleaned_text = cleaned_text.replace(phrase, '')
        cleaned_list.append(cleaned_text)
    return cleaned_list

# Example usage:
list_of_strings = risk_1a
#phrases_to_remove = ["##TABLE_START", "##TABLE_END &#160;", "##TABLE_END", "&#8226;", "&#183;", "FDA&#8217;s"]
phrases_to_remove = ["##TABLE_START", "##TABLE_END &#160;", "##TABLE_END", "&#8226;", "&#183;", "FDA&#8217;s"]#+ list(stop_words)
cleaned_list = remove_phrases_from_list(list_of_strings[:], phrases_to_remove)
#print(cleaned_list)


In [30]:
print(len(cleaned_list))

63340


In [38]:
#break into smaller paragraph the cleaned_list delimeter="&#160;"
def separate_paragraphs_from_list(list_of_paragraphs):
    separated_paragraphs = []
    for paragraph in list_of_paragraphs:
        #smaller_paragraphs = paragraph.split("&#160;")
        smaller_paragraphs = paragraph.split(" \n\n")
        smaller_paragraphs = [p.strip() for p in smaller_paragraphs]
        #separated_paragraphs.extend(smaller_paragraphs)
        separated_paragraphs.append(smaller_paragraphs)
    return separated_paragraphs

# Example usage:
list_of_paragraphs = cleaned_list[:]

result_break = separate_paragraphs_from_list(list_of_paragraphs[:])

In [40]:
len(result_break)

63340

In [42]:
#add result break to the original dataframe df
df.insert(loc=3, column='1a_split', value=result_break)

In [43]:
df.head()

,id,accessionNo,cik,1a_split,ticker,companyName,companyNameLong,formType,description,filedAt,...,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate,10k_risks1A,10k_risks7,10k_risks7A
0,9b58a0b3d70dc9de06bdf3d560b3317a,0000320193-22-000108,320193,"[Item 1A. Risk Factors, The Company&#8217;s bu...",AAPL,Apple Inc.,Apple Inc. (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2022-10-27T18:01:14-04:00,...,https://www.sec.gov/Archives/edgar/data/320193...,"[{'companyName': 'Apple Inc. (Filer)', 'cik': ...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '8', 'description': 'XBRL TAXONO...",[],2022-09-24,NaN,Item 1A. Risk Factors \n\nThe Company&#8217;s...,Item 7. Management&#8217;s Discussion and Ana...,Item 7A. Quantitative and Qualitative Disclos...
1,3249304e3a3d5edb2caddaba70628064,0000320193-21-000105,320193,"[Item 1A. Risk Factors, The Company&#8217;s bu...",AAPL,Apple Inc.,Apple Inc. (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2021-10-28T18:04:28-04:00,...,https://www.sec.gov/Archives/edgar/data/320193...,"[{'fiscalYearEnd': '0925', 'stateOfIncorporati...","[{'sequence': '1', 'size': '2051190', 'documen...","[{'sequence': '8', 'size': '60123', 'documentU...",[],2021-09-25,2021-09-25,Item 1A. Risk Factors \n\nThe Company&#8217;s...,Item 7. Management&#8217;s Discussion and Ana...,Item 7A. Quantitative and Qualitative Disclos...
2,96618433ccb8db908723fb09aa081a34,0000320193-20-000096,320193,"[Item 1A. Risk Factors, The following discussi...",AAPL,Apple Inc.,Apple Inc. (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2020-10-29T18:06:25-04:00,...,https://www.sec.gov/Archives/edgar/data/320193...,"[{'fiscalYearEnd': '0926', 'stateOfIncorporati...","[{'sequence': '1', 'size': '2467306', 'documen...","[{'sequence': '10', 'size': '64821', 'document...",[],2020-09-26,NaN,Item 1A. Risk Factors \n\nThe following discu...,Item 7. Management&#8217;s Discussion and Ana...,Item 7A. Quantitative and Qualitative Disclos...
3,e874e34437539610a5d89da3ff86d018,0000320193-19-000119,320193,"[Item 1A., Risk Factors, The following discuss...",AAPL,Apple Inc.,Apple Inc. (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2019-10-30T18:12:36-04:00,...,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,"[{'companyName': 'Apple Inc. (Filer)', 'cik': ...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '10', 'description': 'XBRL TAXON...",[],2019-09-28,NaN,Item 1A. \n\nRisk Factors \n\n##TABLE_END The...,Item 7. \n\nManagement&#8217;s Discussion and...,Item 7A. \n\nQuantitative and Qualitative Dis...
4,d2f06aa74a77a94550011b4747ccc759,0000320193-18-000145,320193,"[Item 1A., Risk Factors, The following discuss...",AAPL,APPLE INC,APPLE INC (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2018-11-05T08:01:40-05:00,...,https://www.sec.gov/Archives/edgar/data/320193...,"[{'companyName': 'APPLE INC (Filer)', 'cik': '...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '9', 'description': 'XBRL INSTAN...",[],2018-09-29,NaN,Item 1A. \n\nRisk Factors \n\n##TABLE_END The...,Item 7. \n\nManagement&#8217;s Discussion and...,Item 7A. \n\nQuantitative and Qualitative Dis...


In [46]:
#explode the column 1a_split
df=df.explode('1a_split')

In [47]:
df.shape

(9923802, 23)

In [48]:
df.head(2)

,id,accessionNo,cik,1a_split,ticker,companyName,companyNameLong,formType,description,filedAt,...,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate,10k_risks1A,10k_risks7,10k_risks7A
0,9b58a0b3d70dc9de06bdf3d560b3317a,0000320193-22-000108,320193,Item 1A. Risk Factors,AAPL,Apple Inc.,Apple Inc. (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2022-10-27T18:01:14-04:00,...,https://www.sec.gov/Archives/edgar/data/320193...,"[{'companyName': 'Apple Inc. (Filer)', 'cik': ...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '8', 'description': 'XBRL TAXONO...",[],2022-09-24,NaN,Item 1A. Risk Factors \n\nThe Company&#8217;s...,Item 7. Management&#8217;s Discussion and Ana...,Item 7A. Quantitative and Qualitative Disclos...
0,9b58a0b3d70dc9de06bdf3d560b3317a,0000320193-22-000108,320193,"The Company&#8217;s business, reputation, resu...",AAPL,Apple Inc.,Apple Inc. (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2022-10-27T18:01:14-04:00,...,https://www.sec.gov/Archives/edgar/data/320193...,"[{'companyName': 'Apple Inc. (Filer)', 'cik': ...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '8', 'description': 'XBRL TAXONO...",[],2022-09-24,NaN,Item 1A. Risk Factors \n\nThe Company&#8217;s...,Item 7. Management&#8217;s Discussion and Ana...,Item 7A. Quantitative and Qualitative Disclos...


In [52]:
#remove the rows with fewer characters in 1a_split
def count_words(risksplit):
    return len(risksplit.split(" "))
word_count_mask = df['1a_split'].apply(count_words)>20
filtered_df=df[word_count_mask]

In [53]:
filtered_df.head()

,id,accessionNo,cik,1a_split,ticker,companyName,companyNameLong,formType,description,filedAt,...,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate,10k_risks1A,10k_risks7,10k_risks7A
0,9b58a0b3d70dc9de06bdf3d560b3317a,0000320193-22-000108,320193,"The Company&#8217;s business, reputation, resu...",AAPL,Apple Inc.,Apple Inc. (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2022-10-27T18:01:14-04:00,...,https://www.sec.gov/Archives/edgar/data/320193...,"[{'companyName': 'Apple Inc. (Filer)', 'cik': ...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '8', 'description': 'XBRL TAXONO...",[],2022-09-24,NaN,Item 1A. Risk Factors \n\nThe Company&#8217;s...,Item 7. Management&#8217;s Discussion and Ana...,Item 7A. Quantitative and Qualitative Disclos...
0,9b58a0b3d70dc9de06bdf3d560b3317a,0000320193-22-000108,320193,"Because of the following factors, as well as o...",AAPL,Apple Inc.,Apple Inc. (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2022-10-27T18:01:14-04:00,...,https://www.sec.gov/Archives/edgar/data/320193...,"[{'companyName': 'Apple Inc. (Filer)', 'cik': ...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '8', 'description': 'XBRL TAXONO...",[],2022-09-24,NaN,Item 1A. Risk Factors \n\nThe Company&#8217;s...,Item 7. Management&#8217;s Discussion and Ana...,Item 7A. Quantitative and Qualitative Disclos...
0,9b58a0b3d70dc9de06bdf3d560b3317a,0000320193-22-000108,320193,This section should be read in conjunction wit...,AAPL,Apple Inc.,Apple Inc. (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2022-10-27T18:01:14-04:00,...,https://www.sec.gov/Archives/edgar/data/320193...,"[{'companyName': 'Apple Inc. (Filer)', 'cik': ...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '8', 'description': 'XBRL TAXONO...",[],2022-09-24,NaN,Item 1A. Risk Factors \n\nThe Company&#8217;s...,Item 7. Management&#8217;s Discussion and Ana...,Item 7A. Quantitative and Qualitative Disclos...
0,9b58a0b3d70dc9de06bdf3d560b3317a,0000320193-22-000108,320193,The Company&#8217;s operations and performance...,AAPL,Apple Inc.,Apple Inc. (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2022-10-27T18:01:14-04:00,...,https://www.sec.gov/Archives/edgar/data/320193...,"[{'companyName': 'Apple Inc. (Filer)', 'cik': ...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '8', 'description': 'XBRL TAXONO...",[],2022-09-24,NaN,Item 1A. Risk Factors \n\nThe Company&#8217;s...,Item 7. Management&#8217;s Discussion and Ana...,Item 7A. Quantitative and Qualitative Disclos...
0,9b58a0b3d70dc9de06bdf3d560b3317a,0000320193-22-000108,320193,The Company has international operations with ...,AAPL,Apple Inc.,Apple Inc. (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2022-10-27T18:01:14-04:00,...,https://www.sec.gov/Archives/edgar/data/320193...,"[{'companyName': 'Apple Inc. (Filer)', 'cik': ...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '8', 'description': 'XBRL TAXONO...",[],2022-09-24,NaN,Item 1A. Risk Factors \n\nThe Company&#8217;s...,Item 7. Management&#8217;s Discussion and Ana...,Item 7A. Quantitative and Qualitative Disclos...


In [54]:
filtered_df.shape

(5079714, 23)

In [69]:
#reading from the text file of stopwords
lines1=[]
with open('StopWords_GenericLong.txt','r') as file:
    lines1=file.read().split('\n')
lines2=[]
with open('StopWords_DatesandNumbers.txt','r') as file:
    lines2=file.read().lower().split('\n')

In [70]:
stopwords_list_specific=lines1+lines2

In [72]:
def preprocess(doc):
    stop=list('liabilities transactions acquisitions accounts risk risks companys company future factors required tablestart tableend able obtain additional us unable would newcompany will value information years upon company’s fiscal rate based report sales management services form costs related tax ended certain market credit products amount period net including operations securities cash time statements income section common assets shares business plan year date interest december agreement stock may financial million shall also result could results significant including tableendtablestart'.split(" "))
    stop_words = set(stopwords.words("english")+stop+stopwords_list_specific)
    #ps = PorterStemmer()
    words = word_tokenize(doc.lower())
    #words = [ps.stem(word) for word in words if word.isalpha()]
    words = [word for word in words if word not in stop_words]  # Remove stopwords
    return words

#function to perform all the cleaning
import re
def cleanText(text):
    text = re.sub("https?:\/\/.*[\r\n]*", "", text)
    text = re.sub("@\S+", "", text)
    text = re.sub("#", "", text)
    text = text.replace('\n',"")
    text = re.sub(r'[^a-zA-Z ]+',"",text)
    text = text.lower()
    text = text.strip()
    return text

In [56]:
clean_list=[cleanText(i) for i in list(filtered_df['1a_split'])]

In [60]:
#add clean_list as a column to the df
filtered_df.insert(loc=4, column='1a_split_clean', value=clean_list)

In [61]:
#clean_list[:10]

In [62]:
filtered_df.head(2)

,id,accessionNo,cik,1a_split,1a_split_clean,ticker,companyName,companyNameLong,formType,description,...,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate,10k_risks1A,10k_risks7,10k_risks7A
0,9b58a0b3d70dc9de06bdf3d560b3317a,0000320193-22-000108,320193,"The Company&#8217;s business, reputation, resu...",the companys business reputation results of op...,AAPL,Apple Inc.,Apple Inc. (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,...,https://www.sec.gov/Archives/edgar/data/320193...,"[{'companyName': 'Apple Inc. (Filer)', 'cik': ...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '8', 'description': 'XBRL TAXONO...",[],2022-09-24,NaN,Item 1A. Risk Factors \n\nThe Company&#8217;s...,Item 7. Management&#8217;s Discussion and Ana...,Item 7A. Quantitative and Qualitative Disclos...
0,9b58a0b3d70dc9de06bdf3d560b3317a,0000320193-22-000108,320193,"Because of the following factors, as well as o...",because of the following factors as well as ot...,AAPL,Apple Inc.,Apple Inc. (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,...,https://www.sec.gov/Archives/edgar/data/320193...,"[{'companyName': 'Apple Inc. (Filer)', 'cik': ...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '8', 'description': 'XBRL TAXONO...",[],2022-09-24,NaN,Item 1A. Risk Factors \n\nThe Company&#8217;s...,Item 7. Management&#8217;s Discussion and Ana...,Item 7A. Quantitative and Qualitative Disclos...


In [ ]:
#preprocess the 1a_split_clean , ie remove the stopwords 
processed_text= [preprocess(l) for l in list(filtered_df['1a_split_clean'])]

In [ ]:
#add processed text to the filtered_df
filtered_df.insert(loc=5, column='1a_split_finalProcessed', value= processed_text)

In [ ]:
#save processed_text to a df such that it can be directly passed into the lda model
#dff=pd.DataFrame(filtered_df)
filtered_df.to_csv(r'C:\Mousumi_SEC\10K_files\cleanDataframe1ARisk_final.csv')

In [ ]:
filtered_df.shape

In [ ]:
filtered_df.head()